In [8]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from typing import Optional, Type, Callable
from pydantic import Field
import requests
import json

'''
# APIキーをセット (変数名はLangChain側で決められています)
from langchain.llms import OpenAI
open_api_key = os.environ["openai_api_key"]
# 言語モデルを指定
llm = OpenAI(temperature=0)
'''
from langchain.llms import LlamaCpp
import os
llm = LlamaCpp(
    model_path="llama.cpp/models/rinna-youri-7b-chat-q4_K_M.gguf",
    # model_path=r"llama.cpp/cpp\models\llama-2-7b-chat.Q4_K_M.gguf",
    input={
        "max_tokens": 32,
        "stop": ["System:", "User:", "Assistant:", "\n"],
    },
    verbose=True,
    temperature=1,
    n_ctx=2048
)

# WebサーバのAPIエンドポイント
url = "https://cir.nii.ac.jp/opensearch/all?q=%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB&count=5&sortorder=0&format=json"
#"http://localhost:5000/api/"

# トピック一覧を取得するツール
class ListTopicTool(BaseTool): # BaseToolクラスのサブクラスとして、クラスを自作
    name = "ListTopic"
    description = """
    JSONデータを受け取ってtitleをまとめてください
    """
    
    # エンドポイントにGETリクエストを送信
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        response = requests.get(url)   #  + "list_topics")
        topics = response.json()
        return topics
    
    # 非同期実行の関数も必須
    async def _run(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        raise NotImplementedError("ListTopicTool does not support async")

        

def _print_func(text: str) -> None:
    print("\n")
    print(text)

# ユーザにトピック選択を促すツール
class HumanInputTool(BaseTool):
    name = "Human"
    description = """
    If you have obtained topics or if the Human`s desired "topic_name" is ambiguous, 
    you have must ask a Human which topics they are interested in. 
    The input should be a question for the Human. "
    You can also consult a Human when you feel stuck or unsure about what to do next."
    """
    
    # ユーザへの質問を表示する関数
    prompt_func: Callable[[str], None] = Field(default_factory=lambda: _print_func)
    # 入力を受け付ける関数
    input_func: Callable = Field(default_factory=lambda: input)
        
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        # ユーザへの質問を表示する
        self.prompt_func(query)
        # 入力を受け付ける関数を実行
        return self.input_func()

    # 非同期実行関数の定義は必須
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None,) -> str:
        raise NotImplementedError("Human tool does not support async")

    
# 指定トピックの内容を取得するツール
class GetTopicTool(BaseTool):  # BaseToolクラスのサブクラスとして、クラスを自作
    name = "GetTopic"
    description = """
    Retrieve a specific topic requested by the user.
    This functionality is used when the user is seeking the content of a particular topic, 
    such as when they specify a topic name like "Cloud Expansion Strategy" or when they ask a question like "Tell me about skill development for young employees."
    When executing this functionality, you need to provide input in the form of a dictionary with key-value pairs. 
    The key should be "topic_name" in Japanese and the value should be the specified topic name by the user or the extracted topic name from the user's query.
    """
    
    # エンドポイントにPOSTリクエストを送信
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        query = query.replace("'", '"').encode('utf-8')
        query_dict= json.loads(query)
        response = requests.post(url + "get_topic", query_dict)
        topic_content = response.text
        return topic_content
        
    # 非同期実行関数の定義は必須
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        raise NotImplementedError("ListTopicTool does not support async")
            
            
# ツールを設定
tools = [
    ListTopicTool()
]
prompt_func: Callable[[str], None] = Field(default_factory=lambda: _print_func)

    # HumanInputTool(),
    # GetTopicTool()

# エージェントを定義
# agent = initialize_agent(tools, OpenAI(temperature=0), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         handle_parsing_errors=True
                        )

# エージェント起動
agent.run("トピックの一覧を表示し、トピックの内容を把握したいです。")


c:\Users\onowa\miniconda3\envs\vir_env\lib\site-packages\langchain\utils\utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 




> Entering new AgentExecutor chain...
  You should always know how to get from one place to another.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

Llama.generate: prefix-match hit


 I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


 You should always know how to get from one place to another.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

Llama.generate: prefix-match hit


 I now know the final answer
Final Answer: The following topics are available for display:
    - Topic 0

> Finished chain.


'The following topics are available for display:\n    - Topic 0'